In [129]:
with open('input.txt') as f:
    hexString = f.readline()

# hexString = 'C200B40A82'
# hexString = 'EE00D40C823060'
# hexString = 'A0016C880162017C3686B18A3D4780'

hex_to_binary = {
    '0': '0000',
    '1': '0001',
    '2': '0010',
    '3': '0011',
    '4': '0100',
    '5': '0101',
    '6': '0110',
    '7': '0111',
    '8': '1000',
    '9': '1001',
    'A': '1010',
    'B': '1011',
    'C': '1100',
    'D': '1101',
    'E': '1110',
    'F': '1111'
}




In [ ]:
# Step 1 Convert hex to binary:

packet = ''
for c in hexString:
    packet += hex_to_binary[c]

subPackets = []
version_sum = 0


def getVersions(packet):
    # first 3 digits is the version
    v_binary = packet[0: 3]
    v_decimal = int(v_binary, 2)
    print(f'v: {v_decimal}')
    if len(packet) < 11:
        return [0, len(packet)]

    operator_binary = packet[3: 6]
    operator_decimal = int(operator_binary, 2)
    print(f'o: {operator_decimal}')

    binary_string = ''

    if (operator_decimal == 4):
        # literal string
        for i in range(6, len(packet), 5):
            if packet[i] == '1':
                print(f'literal string {packet[i: i+5]}')
                binary_string += packet[i+1: i+5]
            elif packet[i] == '0':
                print(f'literal string {packet[i: i+5]} (end) @ {i+5}')
                binary_string += packet[i+1:i+5]
                break
        last_index = i+5
        # for j in range(i+5, len(packet)):
        #     if packet[j] == '1':
        #         last_index = j
        #         break
        # return int(binary_string, 2)
        print(f'version: {v_decimal}, last_index:  {last_index}')
        print(f'total literal string: {int(binary_string, 2)}')
        return [v_decimal, last_index]
    else:
        # operator
        print(f'length type: {packet[6]}')
        if packet[6] == '0':
            # skip forward 16 bits
            length_binary = packet[7:22]
            length = int(length_binary, 2)
            print(f'length(b): {length_binary}')
            print(f'length: {length}')
            i = 22
            while i < len(packet):
                print(packet[i:])
                v, j = getVersions(packet[i:])
                v_decimal += v
                i += j
            return [v_decimal, i]
        elif packet[6] == '1':
            # skip forward 11 bits
            length_binary = packet[7:18]
            length = int(length_binary, 2)
            print(f'repeat(b): {length_binary}')
            print(f'repeat: {length}')
            i = 18
            print(packet[i:])
            while i < len(packet):
                v, j = getVersions(packet[i:])
                v_decimal += v
                i += j
            return [v_decimal, i]


print(packet)
getVersions(packet)


In [143]:
# Part 2

def convertHexToBinary(hexString):
    packet = ''
    for c in hexString:
        packet += hex_to_binary[c]
    return packet


def evaluateExpression(packet):
    # first 3 digits is the version

    operator_binary = packet[3: 6]
    typeID = int(operator_binary, 2)

    binary_string = ''

    if (typeID == 4):
        # literal string
        for i in range(6, len(packet), 5):
            if packet[i] == '1':
                binary_string += packet[i+1: i+5]
            elif packet[i] == '0':
                binary_string += packet[i+1:i+5]
                break
        last_index = i+5
        out = int(binary_string, 2)
        return [out, last_index]
    else:
        subPacketValues = []
        # operator
        if packet[6] == '0':
            # skip forward 16 bits
            length_binary = packet[7:22]
            length = int(length_binary, 2)
            count = 0
            i = 22
            # while i < len(packet):
            while count < length:
                if len(packet[i:]) > 11:
                    value, j = evaluateExpression(packet[i:])
                    i += j
                    subPacketValues.append(value)
                    count += j
                else:
                    break

        elif packet[6] == '1':
            # skip forward 11 bits
            repeat_binary = packet[7:18]
            repeat = int(repeat_binary, 2)
            i = 18
            count = 0
            # while i < len(packet):
            while count < repeat:
                if len(packet[i:]) >= 11:
                    value, j = evaluateExpression(packet[i:])
                    i += j
                    subPacketValues.append(value)
                    count += 1
                else:
                    break
        
        last_index = i
        if (typeID == 0):
            out = sum(subPacketValues)
        elif (typeID == 1):
            prod = 1
            for v in subPacketValues:
                prod *= v
            out = prod
        elif (typeID == 2):
            out = min(subPacketValues)
        elif (typeID == 3):
            out = max(subPacketValues)
        elif (typeID == 5):
            if subPacketValues[0] > subPacketValues[1]:
                out = 1
            else:
                out = 0
        elif (typeID == 6):
            if subPacketValues[0] < subPacketValues[1]:
                out = 1
            else:
                out = 0
        elif (typeID == 7):
            if subPacketValues[1] == subPacketValues[0]:
                out = 1
            else:
                out = 0
        else:
            raise ('smth went terribly wrong')
        return [out, last_index]


packet = convertHexToBinary(hexString)
evaluateExpression(packet)[0]


10626195124371